In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
KEY = '2105b9f242d47b69fc73f0f2205be048'

def main_table(company):

    print(f'Создаем основную таблицу для: {company}')

    IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={KEY}').json()
    FIS = requests.get(
        f'https://financialmodelingprep.com/api/v3/financials/income-statement/{company}?apikey={KEY}').json()
    BL = requests.get(
        f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={KEY}').json()
    RT = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={KEY}').json()
    CF = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{company}?apikey={KEY}').json()
    EV = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={KEY}').json()
    PF = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{company}?apikey={KEY}').json()
    CQ = requests.get(f'https://financialmodelingprep.com/api/v3/quote/{company}?apikey={KEY}').json()
    FS = requests.get(
        f'https://financialmodelingprep.com/api/v3/financial-statement-full-as-reported/{company}?apikey={KEY}').json()
    KM = requests.get(f'https://financialmodelingprep.com/api/v3/key-metrics/{company}?apikey={KEY}').json()
    CF_TTM = requests.get(
        f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{company}?period=quarter&apikey={KEY}').json()
    BL_TTM = requests.get(
        f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?period=quarter&apikey={KEY}').json()

    financial_income_statement = pd.DataFrame(FIS['financials'])
    income_statement = pd.DataFrame(IS)
    balance = pd.DataFrame(BL)
    balance_TTM = pd.DataFrame(BL_TTM)
    ratios = pd.DataFrame(RT)
    cash_flow = pd.DataFrame(CF)
    cash_flow_TTM = pd.DataFrame(CF_TTM)
    enterprise_value = pd.DataFrame(EV)
    company_profile = pd.DataFrame(PF)
    real_time_quote = pd.DataFrame(CQ)
    financial_statement = pd.DataFrame(FS)
    key_metrics = pd.DataFrame(KM)

    # Добавление CF за последние 4 месяца TTF

    q_cash_flow_statement = cash_flow_TTM.set_index('date').iloc[:4]
    q_cash_flow_statement = cash_flow_TTM.apply(pd.to_numeric, errors='coerce')
    ttm_cash_flow_statement = cash_flow_TTM[0:4].sum()  # sum up last 4 quarters to get TTM cash flow

    cash_flow_statement = cash_flow[::-1].append(ttm_cash_flow_statement.rename('TTM')).drop(['netIncome'], axis=1)
    final_cash_flow_statement = cash_flow_statement[::-1].reset_index(drop=True)
    # cash_flow - указать в рабочей таблице, что бы венуться к значениям без TTM
    # ++++++++++++++++++++++++++++++++++++++++++ Учет кэша с долгом +++++++++++++++++++++++++++++++++++++++++++++++

    balance_TTM = pd.DataFrame(BL_TTM).iloc[:4]

    totalDebt_TTM = balance_TTM['totalDebt'][0]
    cash_TTM = balance_TTM['cashAndShortTermInvestments'][0]

    work_table = pd.concat(
        (income_statement[['date', 'revenue', 'operatingIncome', 'netIncome', 'eps', 'ebitda',
                           'depreciationAndAmortization', 'interestExpense', 'incomeBeforeTax',
                           'incomeTaxExpense', 'researchAndDevelopmentExpenses', 'generalAndAdministrativeExpenses',
                           'sellingAndMarketingExpenses', 'otherExpenses', 'operatingExpenses', 'costAndExpenses',
                           'grossProfit', ]][0:10],
         balance[['totalAssets', 'totalLiabilities', 'totalStockholdersEquity',
                  'totalDebt', 'netDebt', 'longTermDebt']][0:10],
         ratios[
             ['dividendPayoutRatio', 'dividendYield', 'effectiveTaxRate', 'payoutRatio', 'returnOnAssets']][0:10],
         final_cash_flow_statement[['freeCashFlow', 'dividendsPaid', 'operatingCashFlow']][0:10],
         enterprise_value[['numberOfShares', 'stockPrice']][0:10],
         key_metrics[['debtToEquity', 'bookValuePerShare', 'roe', 'currentRatio']][0:10],
         financial_income_statement[['Dividend per Share', 'Gross Margin']][0:10],
         company_profile[['mktCap', 'sector']][0:10]

         ), axis=1).fillna(0)

    work_table['price'] = real_time_quote['price'].fillna(0)[0]
    work_table['beta'] = company_profile['beta'].fillna(0)[0]
    work_table['totalDebt_TTM'] = balance_TTM['totalDebt'][0]
    work_table['cash_TTM'] = balance_TTM['cashAndShortTermInvestments'][0]


    # последний год с данными
    year_len = 0

    for row in work_table['date']:
        if row != 0:
            year_len += 1

    work_table['year_len'] = year_len

    # ___________________________________ ROE_DuPont _______________________________________________

    Profitability = []
    TechnicalEfficiency = []
    FinancialStructure = []

    for year in range(0, year_len - 1):
        last_year = year + 1
        Profitability.append(work_table['netIncome'][year] / work_table['revenue'][year])
        TechnicalEfficiency.append(work_table['revenue'][year] / (
                (work_table['totalAssets'][year] + work_table['totalAssets'][last_year]) / 2))
        FinancialStructure.append(
            ((work_table['totalAssets'][year] + work_table['totalAssets'][last_year]) / 2) /
            ((work_table['totalStockholdersEquity'][year] +
              work_table['totalStockholdersEquity'][last_year]) / 2))

    table = pd.DataFrame()
    table['Profitability'] = Profitability
    table['TechnicalEfficiency'] = TechnicalEfficiency
    table['FinancialStructure'] = FinancialStructure

    work_table['ROE_DuPont'] = 0

    work_table['ROE_DuPont'] = round((table['FinancialStructure'].fillna(0) * table['TechnicalEfficiency'].fillna(0) * \
                               table['Profitability']).mean(axis=0), 5)


    return work_table